In [1]:
import mpmath as mp
import sympy as sp
sp.init_printing()

Define our parameters
---------------------

In [2]:
precision = 40

A = sp.Integer(12)
i = 43

# A-1 / A+1 will have type sp.Rational. 
# Here we take a Float to given precision
α = ((A-1)/(A+1)).evalf(precision)**2
X = sp.log(1/α)
ξ = 1 + (α/(1-α))*sp.log(α)
u = sp.log(2E6/1)

In [3]:
%whos

Variable    Type       Data/Info
--------------------------------
A           Integer    12
X           Float      0.33410816932633238398308<...>2720037860996712218164558
i           int        43
mp          module     <module 'mpmath' from '/h<...>ages/mpmath/__init__.py'>
precision   int        50
sp          module     <module 'sympy' from '/ho<...>kages/sympy/__init__.py'>
u           Float      14.5086577385242
α           Float      0.71597633136094674556213<...>7751479289940828402366864
ξ           Float      0.15776898982320378204263<...>2101571225404121283376843


Define symbolic solution
------------------------

In [4]:
def n_average_collisions(α, X, u, i):    
    k, m = sp.symbols("k, m")
    R_k = sp.Sum(
        # Summand
        (-1)**m * sp.factorial(k)/sp.factorial(m) * α**m*(1-α)**(k-m+1) * (u-k*X)**m,
        # Limits
        (m, 0, k-1)
    )

    R = sp.Piecewise(
        # R(k>0):
        ((R_k + (-1)**k*α**k*(u-k*X)**k) / (sp.factorial(k)*α**k*(1-α)**k), k > 0), 
        # R(k<=0):
        (1, k <= 0)
    )
    
    N = sp.Sum(
        # Summand
        1 + 1/α * (R*sp.exp((α*u-k*X)/(1-α)) - 1),
        # Limits
        (k, 0, i)
    )
    
    return N

Print analytical result
-----------------------

In [5]:
symbols = sp.symbols("α,X,u,i")
expr = n_average_collisions(*symbols)
expr

      i                                                                       
_____________                                                                 
╲                                                                             
 ╲            ⎧                       ⎛                       k - 1           
  ╲           ⎪                       ⎜                        ____           
   ╲          ⎪                       ⎜                        ╲              
    ╲         ⎪                       ⎜                         ╲       m  m  
     ╲        ⎪          -k        -k ⎜    k  k           k      ╲  (-1) ⋅α ⋅(
      ╲       ⎪         α  ⋅(1 - α)  ⋅⎜(-1) ⋅α ⋅(-X⋅k + u)  +    ╱  ──────────
       ╲      ⎪                       ⎜                         ╱             
        ╲     ⎪                       ⎜                        ╱              
         ╲    ⎪                       ⎜                        ‾‾‾‾           
          ╲   ⎪                       ⎝             

Evaluate result using mpmath
----------------------------
Passing in `n` implicitly uses mpmath, but we need to specify mpmath types for the constants

In [6]:
expr.evalf(n=precision, subs={symbols[0]: α,
                              symbols[1]: X,
                              symbols[2]: u,
                              symbols[3]: i})

---

Alternatively, using a direct approach
--------------------------------------

In [7]:
import mpmath as mp
mp.mp.dps = precision

def n_average_collisions_numerical(α, X, u, i):    
    def term(k):
        return 1 + 1/α * (R(α, X, u, k)*mp.exp((α*u-k*X)/(1-α)) - 1)
    return mp.nsum(term, [0, i])

def R(α, X, u, k):
    if not k:
        return 1.0
    
    def term(m):
        return (-1)**m * mp.factorial(k)/mp.factorial(m) * α**m*(1-α)**(k-m+1) * (u-k*X)**m 
              
    return (mp.nsum(term, [0, k-1]) + (-1)**k*α**k*(u-k*X)**k) / (mp.factorial(k)*α**k*(1-α)**k)

Evaluate result, using mpmath types 
-----------------------------------
These types implement operator overloading, so behave numerically

In [8]:
n_average_collisions_numerical(α, X, u, 43)

mpf('92.647226612382042830370553040941674860743773611584725')